In [7]:
import numpy as np
import pandas as pd  # для работы с данными
import time  # для оценки времени
import torch  # для написания нейросети
from torch import nn
import re

from torch.utils.data import TensorDataset, DataLoader

In [2]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cuda:0'

# Задание 1

In [3]:
#функция генерации последователльности
def get_seq(d=10,n=10000):
    X = []
    Y = []
    for i in range(n):
        X.append(np.random.randint(0,10,d))
        x1 = X[i][0]
        y = [ (x + x1) if (x + x1) < 10 else (x + x1 -10) for x in X[i]]
        y[0]= x1
        Y.append(np.array(y))
    return X,Y

In [4]:
BATCH_SIZE = 32
SEQ_LEN = 10
train_data = get_seq(d=SEQ_LEN,n=100000)
train_dataset = TensorDataset(torch.Tensor(train_data[0]).long(),torch.Tensor(train_data[1]).long())
train = DataLoader(train_dataset,batch_size=BATCH_SIZE)

test_data = get_seq(d=SEQ_LEN,n=10000)
test_dataset = TensorDataset(torch.Tensor(test_data[0]).long(),torch.Tensor(test_data[1]).long())
test = DataLoader(test_dataset,batch_size=BATCH_SIZE)

C:\Users\RK\AppData\Local\Temp\ipykernel_19016\3191984709.py:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:277.)
  train_dataset = TensorDataset(torch.Tensor(train_data[0]).long(),torch.Tensor(train_data[1]).long())


In [7]:
train_data[0][0]

array([9, 7, 9, 9, 8, 0, 8, 7, 2, 0])

In [8]:
train_data[1][0]

array([9, 6, 8, 8, 7, 9, 7, 6, 1, 9])

In [9]:
#функция тренировки n_ex - количество вывводимых результатов из тестовой выборки
def train_model(model,n_epoch = 10,nshow=10,n_ex = 10):
    for ep in range(n_epoch):
        train_iters, train_passed  = 0, 0
        train_loss, train_acc = 0., 0.
        start=time.time()

        model.train()
        for X, y in train:
            if X.shape[0] !=BATCH_SIZE:
                continue

            X, y = X.to(device), y.to(device)

            optimizer.zero_grad()
            y_pred = model(X)
            y_pred = y_pred.view(BATCH_SIZE * seq_len, n_chars)
            real_y = y.view(BATCH_SIZE * seq_len).long()
            l = loss(y_pred, real_y)
            l.backward()
            optimizer.step()
            train_loss += l.item()
            train_acc += (y_pred.argmax(dim=1) == real_y).sum().item()/seq_len
            train_iters += 1
            train_passed += len(X)

        
        test_iters, test_passed  = 0, 0
        test_loss, test_acc = 0., 0.

        model.eval()
        for X, y in test:
            if X.shape[0]!=BATCH_SIZE:
                continue
            X, y = X.to(device), y.to(device)
            y_pred = model(X)
            y_pred = y_pred.view(BATCH_SIZE * seq_len, n_chars)
            real_y = y.view(BATCH_SIZE * seq_len).long()
            l = loss(y_pred, real_y)
            test_loss += l.item()
            test_acc += (y_pred.argmax(dim=1) == real_y).sum().item()/seq_len
            test_iters += 1
            test_passed += len(X)

        if ep%nshow==0:
            print("ep: {}, taked: {:.3f}, train_loss: {}, train_acc: {}, test_loss: {}, test_acc: {}".format(
                ep, time.time() - start, round(train_loss / train_iters,5), round(train_acc / train_passed,5),
                round(test_loss / test_iters,5), round(test_acc / test_passed,5))
                
            )
            
    return y_pred.argmax(dim=1).view(BATCH_SIZE,-1)[:n_ex],real_y.view(BATCH_SIZE,-1)[:n_ex]


#функция для тренировки разных моделей
#выводит примеры по 20 символов
def m_train(rnn='RNN',lr = 0.01):
    model = Network()
    if rnn == 'LSTM':
        model.rnn = nn.LSTM(embeding_n, hidden_n,batch_first=True)
    elif rnn == 'GRU':
        model.rnn = nn.GRU(embeding_n, hidden_n,batch_first=True)
    else:
        if rnn !='RNN':
            raise 
    model.to(device)
    global loss 
    loss = torch.nn.CrossEntropyLoss()  # типичный лосс многоклассовой классификации
    global optimizer 
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    test_predict,test_real = train_model(model,n_epoch = 10,nshow=1,n_ex = 5)
    
    print('#####################')
    print('test predict')
    print('\n'.join([' '.join(map(str,test_predict[i][:20].to('cpu').numpy())) for i in range(5)]))
    print('test predict')
    print('\n'.join([' '.join(map(str,test_real[i][:20].to('cpu').numpy())) for i in range(5)]))
    
    return test_predict,test_real

In [10]:
seq_len = SEQ_LEN
hidden_n = 64
n_chars = 10
embeding_n = 32

class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.embedding = torch.nn.Embedding(n_chars, embeding_n)
        self.rnn = torch.nn.RNN(embeding_n, hidden_n,batch_first=True)#num_layers=2
        self.out = torch.nn.Linear(hidden_n, n_chars)

    def forward(self, sentences, state=None):
        x = self.embedding(sentences) 
        x, s = self.rnn(x) # берём выход с последнего слоя для всех токенов, а не скрытое состояние
        return self.out(x)

### RNN

In [11]:
predict, real = m_train(rnn='RNN',lr = 0.01)

ep: 0, taked: 6.509, train_loss: 0.26054, train_acc: 0.90443, test_loss: 0.15136, test_acc: 0.92481
ep: 1, taked: 7.359, train_loss: 0.00903, train_acc: 0.99741, test_loss: 0.00052, test_acc: 1.0
ep: 2, taked: 7.847, train_loss: 0.23219, train_acc: 0.90892, test_loss: 0.096, test_acc: 0.96173
ep: 3, taked: 7.690, train_loss: 0.08532, train_acc: 0.978, test_loss: 0.00641, test_acc: 0.99959
ep: 4, taked: 7.050, train_loss: 0.00212, train_acc: 0.9999, test_loss: 0.0007, test_acc: 0.99997
ep: 5, taked: 7.376, train_loss: 0.01325, train_acc: 0.99759, test_loss: 0.39001, test_acc: 0.90636
ep: 6, taked: 6.994, train_loss: 0.05893, train_acc: 0.98144, test_loss: 0.00085, test_acc: 1.0
ep: 7, taked: 7.454, train_loss: 0.09585, train_acc: 0.98127, test_loss: 0.18978, test_acc: 0.95879
ep: 8, taked: 7.386, train_loss: 0.04615, train_acc: 0.99101, test_loss: 0.0042, test_acc: 0.99979
ep: 9, taked: 6.870, train_loss: 0.16192, train_acc: 0.9619, test_loss: 0.09636, test_acc: 0.97777
################

### LSTM

In [12]:
predict, real = m_train(rnn='LSTM',lr = 0.01)

ep: 0, taked: 7.915, train_loss: 0.05171, train_acc: 0.983, test_loss: 5e-05, test_acc: 1.0
ep: 1, taked: 7.782, train_loss: 2e-05, train_acc: 1.0, test_loss: 1e-05, test_acc: 1.0
ep: 2, taked: 7.959, train_loss: 0.0, train_acc: 1.0, test_loss: 0.0, test_acc: 1.0
ep: 3, taked: 8.004, train_loss: 0.0, train_acc: 1.0, test_loss: 0.0, test_acc: 1.0
ep: 4, taked: 7.872, train_loss: 0.0, train_acc: 1.0, test_loss: 0.0, test_acc: 1.0
ep: 5, taked: 7.972, train_loss: 0.00188, train_acc: 0.99964, test_loss: 3e-05, test_acc: 1.0
ep: 6, taked: 7.916, train_loss: 2e-05, train_acc: 1.0, test_loss: 1e-05, test_acc: 1.0
ep: 7, taked: 8.095, train_loss: 0.0, train_acc: 1.0, test_loss: 0.0, test_acc: 1.0
ep: 8, taked: 7.984, train_loss: 0.0, train_acc: 1.0, test_loss: 0.0, test_acc: 1.0
ep: 9, taked: 7.911, train_loss: 0.0, train_acc: 1.0, test_loss: 0.0, test_acc: 1.0
#####################
test predict
7 3 2 1 8 1 9 9 4 6
8 3 9 4 9 8 3 2 1 9
3 6 4 4 4 6 7 8 0 1
3 7 8 4 2 5 6 0 6 6
4 1 0 3 5 5 6 8 8 7

### GRU

In [13]:
predict, real = m_train(rnn='GRU',lr = 0.01)

ep: 0, taked: 7.758, train_loss: 0.05913, train_acc: 0.98087, test_loss: 0.0001, test_acc: 1.0
ep: 1, taked: 7.886, train_loss: 4e-05, train_acc: 1.0, test_loss: 1e-05, test_acc: 1.0
ep: 2, taked: 7.827, train_loss: 1e-05, train_acc: 1.0, test_loss: 0.0, test_acc: 1.0
ep: 3, taked: 7.865, train_loss: 0.0, train_acc: 1.0, test_loss: 0.0, test_acc: 1.0
ep: 4, taked: 7.848, train_loss: 0.0, train_acc: 1.0, test_loss: 0.0, test_acc: 1.0
ep: 5, taked: 7.813, train_loss: 0.0, train_acc: 1.0, test_loss: 0.0, test_acc: 1.0
ep: 6, taked: 7.875, train_loss: 0.0, train_acc: 1.0, test_loss: 0.0, test_acc: 1.0
ep: 7, taked: 7.840, train_loss: 0.00198, train_acc: 0.99948, test_loss: 3e-05, test_acc: 1.0
ep: 8, taked: 7.838, train_loss: 1e-05, train_acc: 1.0, test_loss: 1e-05, test_acc: 1.0
ep: 9, taked: 7.806, train_loss: 0.0, train_acc: 1.0, test_loss: 0.0, test_acc: 1.0
#####################
test predict
7 3 2 1 8 1 9 9 4 6
8 3 9 4 9 8 3 2 1 9
3 6 4 4 4 6 7 8 0 1
3 7 8 4 2 5 6 0 6 6
4 1 0 3 5 5 6 

## Увеличим длину последовательности

In [14]:
BATCH_SIZE = 32

SEQ_LEN = 100
train_data = get_seq(d=SEQ_LEN,n=100000)
train_dataset = TensorDataset(torch.Tensor(train_data[0]).long(),torch.Tensor(train_data[1]).long())
train = DataLoader(train_dataset,batch_size=BATCH_SIZE)

test_data = get_seq(d=SEQ_LEN,n=10000)
test_dataset = TensorDataset(torch.Tensor(test_data[0]).long(),torch.Tensor(test_data[1]).long())
test = DataLoader(test_dataset,batch_size=BATCH_SIZE)

seq_len = SEQ_LEN

### RNN

In [17]:
predict, real = m_train(rnn='RNN',lr = 0.01)

ep: 0, taked: 10.007, train_loss: 1.49861, train_acc: 0.28649, test_loss: 1.33623, test_acc: 0.31411
ep: 1, taked: 9.176, train_loss: 1.14976, train_acc: 0.38704, test_loss: 1.09756, test_acc: 0.40836
ep: 2, taked: 9.390, train_loss: 1.21723, train_acc: 0.4055, test_loss: 1.23173, test_acc: 0.39458
ep: 3, taked: 9.330, train_loss: 0.87714, train_acc: 0.49247, test_loss: 0.80032, test_acc: 0.51067
ep: 4, taked: 9.351, train_loss: 1.0137, train_acc: 0.46415, test_loss: 1.52717, test_acc: 0.30636
ep: 5, taked: 9.295, train_loss: 1.10144, train_acc: 0.42205, test_loss: 0.79715, test_acc: 0.51531
ep: 6, taked: 9.303, train_loss: 0.66455, train_acc: 0.63039, test_loss: 0.60956, test_acc: 0.60822
ep: 7, taked: 9.371, train_loss: 0.59012, train_acc: 0.64095, test_loss: 0.49523, test_acc: 0.70826
ep: 8, taked: 9.394, train_loss: 1.09733, train_acc: 0.449, test_loss: 1.35005, test_acc: 0.34334
ep: 9, taked: 9.370, train_loss: 1.07817, train_acc: 0.45406, test_loss: 1.20661, test_acc: 0.41846
###

### LSTM

In [18]:
predict, real = m_train(rnn='LSTM',lr = 0.01)

ep: 0, taked: 9.754, train_loss: 0.38723, train_acc: 0.85242, test_loss: 0.00022, test_acc: 1.0
ep: 1, taked: 9.682, train_loss: 0.00141, train_acc: 0.9998, test_loss: 0.00014, test_acc: 1.0
ep: 2, taked: 9.682, train_loss: 4e-05, train_acc: 1.0, test_loss: 1e-05, test_acc: 1.0
ep: 3, taked: 9.634, train_loss: 1e-05, train_acc: 1.0, test_loss: 0.0, test_acc: 1.0
ep: 4, taked: 9.676, train_loss: 0.0, train_acc: 1.0, test_loss: 0.0, test_acc: 1.0
ep: 5, taked: 9.676, train_loss: 0.0, train_acc: 1.0, test_loss: 0.0, test_acc: 1.0
ep: 6, taked: 9.697, train_loss: 0.0, train_acc: 1.0, test_loss: 0.0, test_acc: 1.0
ep: 7, taked: 9.675, train_loss: 0.0, train_acc: 1.0, test_loss: 0.0, test_acc: 1.0
ep: 8, taked: 9.703, train_loss: 0.00081, train_acc: 0.9999, test_loss: 3e-05, test_acc: 1.0
ep: 9, taked: 9.680, train_loss: 1e-05, train_acc: 1.0, test_loss: 0.0, test_acc: 1.0
#####################
test predict
2 7 2 8 5 8 5 0 4 7 8 5 4 7 6 5 1 3 5 8
0 2 6 6 6 6 6 4 3 7 5 0 9 3 6 9 3 2 7 2
6 2 4

### GRU

In [19]:
predict, real = m_train(rnn='GRU',lr = 0.01)

ep: 0, taked: 9.607, train_loss: 0.16705, train_acc: 0.93602, test_loss: 0.00024, test_acc: 1.0
ep: 1, taked: 9.634, train_loss: 0.0001, train_acc: 1.0, test_loss: 3e-05, test_acc: 1.0
ep: 2, taked: 9.544, train_loss: 2e-05, train_acc: 1.0, test_loss: 1e-05, test_acc: 1.0
ep: 3, taked: 9.610, train_loss: 0.0, train_acc: 1.0, test_loss: 0.0, test_acc: 1.0
ep: 4, taked: 9.606, train_loss: 0.0, train_acc: 1.0, test_loss: 0.0, test_acc: 1.0
ep: 5, taked: 9.580, train_loss: 0.0, train_acc: 1.0, test_loss: 0.0, test_acc: 1.0
ep: 6, taked: 9.675, train_loss: 0.00493, train_acc: 0.99906, test_loss: 0.00055, test_acc: 1.0
ep: 7, taked: 9.739, train_loss: 0.00017, train_acc: 1.0, test_loss: 5e-05, test_acc: 1.0
ep: 8, taked: 9.674, train_loss: 3e-05, train_acc: 1.0, test_loss: 1e-05, test_acc: 1.0
ep: 9, taked: 9.594, train_loss: 1e-05, train_acc: 1.0, test_loss: 0.0, test_acc: 1.0
#####################
test predict
2 7 2 8 5 8 5 0 4 7 8 5 4 7 6 5 1 3 5 8
0 2 6 6 6 6 6 4 3 7 5 0 9 3 6 9 3 2 7 2


ВЫВОДЫ: 
на маленькой длине последовательности все рассмотренные модели справились хорошо, а вот с увеличением длины 
обычная RNN уже не смогла сделать верные предсказания, LTSM и  GRU справились с этим хорошо. 
Такое поведения моделей соответствут их архитектуре.

# Задание 2 (дополнительное, из лекции)

In [21]:
import re
import random

In [22]:
with open('nietzsche.txt', encoding='utf-8') as f:
    text = f.read().lower()
print('length:', len(text))
text = re.sub('[^a-z ]', ' ', text)
text = re.sub('\s+', ' ', text)

length: 597844


In [23]:
INDEX_TO_CHAR = sorted(list(set(text)))
CHAR_TO_INDEX = {c: i for i, c in enumerate(INDEX_TO_CHAR)}

In [24]:
MAX_LEN = 40
STEP = 3
SENTENCES = []
NEXT_CHARS = []
for i in range(0, len(text) - MAX_LEN, STEP):
    SENTENCES.append(text[i: i + MAX_LEN])
    NEXT_CHARS.append(text[i + MAX_LEN])
print('Num sents:', len(SENTENCES))

Num sents: 193074


In [25]:
print('Vectorization...')
X = torch.zeros((len(SENTENCES), MAX_LEN), dtype=int)
Y = torch.zeros((len(SENTENCES)), dtype=int)
for i, sentence in enumerate(SENTENCES):
    for t, char in enumerate(sentence):
        X[i, t] = CHAR_TO_INDEX[char]
    Y[i] = CHAR_TO_INDEX[NEXT_CHARS[i]]

Vectorization...


In [68]:
BATCH_SIZE=64
dataset = torch.utils.data.TensorDataset(X, Y)
data = torch.utils.data.DataLoader(dataset, BATCH_SIZE, shuffle=True)

In [27]:
def sample(preds):
    softmaxed = torch.softmax(preds, 0)
    probas = torch.distributions.multinomial.Multinomial(1, softmaxed).sample()
    return probas.argmax()

def generate_text():
    start_index = random.randint(0, len(text) - MAX_LEN - 1)

    generated = ''
    sentence = text[start_index: start_index + MAX_LEN]
    generated += sentence

    for i in range(MAX_LEN):
        x_pred = torch.zeros((1, MAX_LEN), dtype=int)
        for t, char in enumerate(generated[-MAX_LEN:]):
            x_pred[0, t] = CHAR_TO_INDEX[char]

        preds = model(x_pred.cuda()).cpu()
        next_char = INDEX_TO_CHAR[sample(preds)]
        generated = generated + next_char

    print(generated[:MAX_LEN] + '|' + generated[MAX_LEN:])

In [28]:
class NeuralNetwork(nn.Module):
    def __init__(self, rnnClass, dictionary_size, embedding_size, num_hiddens, num_classes):
        super().__init__()
        
        self.num_hiddens = num_hiddens
        self.embedding = nn.Embedding(dictionary_size, embedding_size)
        self.hidden = rnnClass(embedding_size, num_hiddens, batch_first=True)
        self.output = nn.Linear(num_hiddens, num_classes)
        
    def forward(self, X):
        out = self.embedding(X)
        _, state = self.hidden(out)
        predictions = self.output(state[0].squeeze())
        return predictions

In [29]:
model = NeuralNetwork(nn.LSTM, len(CHAR_TO_INDEX), 64, 128, len(CHAR_TO_INDEX))

In [30]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [31]:
model = model.cuda()
for ep in range(100):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    model.train()
    for X_b, y_b in data:
        X_b, y_b = X_b.cuda(), y_b.cuda()
        optimizer.zero_grad()
        answers = model(X_b)
        loss = criterion(answers, y_b)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
    model.eval()
    generate_text()

Epoch 0. Time: 10.187, Train loss: 1.858
re appearing they must still more be clo|gring whad has be gestengr of the form a
Epoch 1. Time: 10.434, Train loss: 1.548
bundant enjoyment even in one s own suff|eriad of miscents for than philolophyphi
Epoch 2. Time: 10.355, Train loss: 1.446
an inward self contempt seek to get out |not a displisionance spirately classurar
Epoch 3. Time: 10.824, Train loss: 1.386
the populace eat and drink and even wher|e worads santing i faith of netherso see
Epoch 4. Time: 10.635, Train loss: 1.344
 against sympathy but to repeat it once |are instance the individual your sensati
Epoch 5. Time: 10.667, Train loss: 1.313
s must when compared with his own flying| which intoullen sout the love have prew
Epoch 6. Time: 10.625, Train loss: 1.289
st hesitation for instance the nobleman |in scientificed bust is old being of eff
Epoch 7. Time: 10.651, Train loss: 1.269
t of his head the contrary of this monst|rowosphere to men you has us mud civiliz
Epoch 8. Time: 1

Epoch 67. Time: 7.571, Train loss: 1.025
as the peculiarly immoral act so that to| them it is make the general converence 
Epoch 68. Time: 7.582, Train loss: 1.025
ays to replace commanders by the summing| hardly as see for the selition of chris
Epoch 69. Time: 7.594, Train loss: 1.025
can descend into the nether world of eve|ry surpression which applies printurity 
Epoch 70. Time: 7.569, Train loss: 1.025
 dealing with them whereas they all rais|es of europe the man any evolution who i
Epoch 71. Time: 7.587, Train loss: 1.025
inces courage directness endurance and n|amure mediocre is evalent no tradary wop
Epoch 72. Time: 7.549, Train loss: 1.023
the free the ever freer spirit to begin |and henum may become himself tead this s
Epoch 73. Time: 7.571, Train loss: 1.024
 same new conditions under which on an a|s others by the theologian to the questi
Epoch 74. Time: 7.553, Train loss: 1.024
rman depth the only thing necessary for |astumental hent through a thing envy a p
Epoch 75. Time: 